In [1]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 24.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 72.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 53.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.4 MB/s eta 0:00:00a 0:00:

In [2]:
from datasets import load_from_disk
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [3]:
# 허깅페이스에서 이미 업로드 된 모델을 불러오는 것도 가능.
# llm = LLM(model="iamjoon/qwen2-7b-rag-ko-checkpoint-141")
llm = LLM(model="./output_dir")

INFO 03-15 16:13:55 __init__.py:207] Automatically detected platform cuda.
INFO 03-15 16:14:01 config.py:549] This model supports multiple tasks: {'generate', 'embed', 'reward', 'score', 'classify'}. Defaulting to 'generate'.
INFO 03-15 16:14:01 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='./output_dir', speculative_config=None, tokenizer='./output_dir', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_n

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 03-15 16:14:08 model_runner.py:1115] Loading model weights took 14.2487 GB
INFO 03-15 16:14:10 worker.py:267] Memory profiling takes 2.42 seconds
INFO 03-15 16:14:10 worker.py:267] the current vLLM instance can use total_gpu_memory (79.14GiB) x gpu_memory_utilization (0.90) = 71.22GiB
INFO 03-15 16:14:10 worker.py:267] model weights take 14.25GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 4.35GiB; the rest of the memory reserved for KV Cache is 52.53GiB.
INFO 03-15 16:14:11 executor_base.py:111] # cuda blocks: 61477, # CPU blocks: 4681
INFO 03-15 16:14:11 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 30.02x
INFO 03-15 16:14:17 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.19it/s]

INFO 03-15 16:14:33 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.19 GiB
INFO 03-15 16:14:33 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 25.39 seconds


In [4]:
test_dataset = load_from_disk('test_dataset')

In [6]:
model = './output_dir'
tokenizer = AutoTokenizer.from_pretrained(model)

prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [7]:
prompt_lst[20]

'<|im_start|>system\n당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: “인재 제일의 삼성이라더니 너무 실망입니다.” 김정미 제일모직 상무(43·사진)가 21년 전 삼성 본관 25층 인사팀의 방명록에 쓴 문장이다. 당시 인사 담당자는 삼성 입사 원서를 받으려고 인사팀을 찾은 그를 힐끗 쳐다보고 말했다. “군필이에요?” 얼굴이 붉어졌지만 김 상무는 지지 않고 대꾸했다. “저는 면제예요. 원서 주세요.”하지만 지원서를 받지 못했다. 여대생이라는 이유에서 거부당한 것이다. 불과 20여년 전, 삼성도 그랬다. “방명록에 학점이나 써놓고 가라”는 직원의 말에 김 상무는 화가 나서 “(삼성에) 실망했다”는 글을 남긴 채 그곳을 나왔다.지난달 30일 저녁 서울 서초동 삼성전자 본사 3층 대회의실에서 20~30대 여성 100여명이 김 상무의 얘기에 귀를 기울였다. 삼성이 여성 임직원의 경험을 외부와 공유하기 위해 만든 ‘여기(女氣) 모여라’ 현장이다. 삼성 트위터, 페이스북, 카카오플러스 등 소셜네트워크서비스(SNS)를 통해 일반인 신청을 받아 초청했다. 이번 강연엔 1000명 넘게 신청이 몰려 10 대 1의 경쟁률을 기록했다. 김 상무는 

In [8]:
label_lst[20]

'\n여대생이라는 이유로 지원서를 받지 못한 인물은 김정미 제일모직 상무입니다. 김정미 상무는 21년 전 삼성 본관 인사팀을 찾아가 입사 원서를 요청했으나, 여대생이라는 이유로 거부당했습니다. 당시 인사 담당자는 "군필이에요?"라고 물었고, 김 상무가 "저는 면제예요. 원서 주세요."라고 대답했지만, 결국 지원서를 받지 못했습니다. [[ref1]]<|im_end|>\n'

In [9]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["<|im_end|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
)

# 생성
outputs = llm.generate(prompt_lst[0], sampling_params)

# 결과 출력
for output in outputs:
    print()
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it, est. speed input: 2708.17 toks/s, output: 69.24 toks/s]



1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 지난달 배추의 평균 판매가는 1402원으로, 10월보다 31.1% 하락했고, 1월(1886원)보다는 13.7% 내렸습니다 [[ref1]].


In [10]:
outputs = llm.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 941/941 [04:56<00:00,  3.18it/s, est. speed input: 11883.65 toks/s, output: 739.46 toks/s]


In [14]:
for output, label in zip(outputs[:5], label_lst[:5]):
    print('예측:', output.outputs[0].text)
    print('레이블:', label)
    print('--')

예측: 
1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 지난달 배추의 평균 판매가는 1402원으로, 10월보다 31.1% 하락했고, 1월(1886원)보다는 13.7% 내렸습니다 [[ref1]].
레이블: 
1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 배추 한 포기(1.95㎏)의 지난달 평균 판매가는 1402원으로, 이는 1월(1886원)보다 13.7% 내린 가격입니다 [[ref1]].<|im_end|>

--
예측: 
송파구청장이 사법고시에 합격한 연도는 2002년입니다. [[ref3]]
레이블: 
송파구청장 박춘희는 2002년에 사법시험에 합격했습니다. [[ref3]]<|im_end|>

--
예측: 
샤인 머스캣의 별명은 "망고 포도"입니다. 샤인 머스캣은 청포도의 일종으로, 씹을수록 망고와 같은 향이나 '망고 포도'로 불리며, 일반 포도보다 당도는 높고 산도는 낮아 고당도의 맛과 향이 특징입니다 [[ref4]].
레이블: 
샤인 머스캣의 별명은 "망고 포도"입니다. 이는 샤인 머스캣이 씹을수록 망고와 같은 향이 나기 때문에 붙여진 별명입니다. 샤인 머스캣은 청포도의 일종으로, 일반 포도보다 당도가 높고 산도는 낮아 고당도의 맛과 향이 특징이며, 씨가 없어 먹기가 편리해 최근 소비자들에게 높은 인기를 얻고 있습니다 [[ref4]].<|im_end|>

--
예측: 
발베니 스토리 전시회에 도서 300여 권을 제공하는 서점이 있는 도시는 서울입니다. 서울 연희동에 위치한 독립 서점 '밤의 서점'이 300여 권의 도서를 발베니 스토리 컬렉션 4가지 제품의 키워드와 연관된 책으로 큐레이션해 전시회에 제공하였습니다 [[ref5]].
레이블: 
발베니 스토리 전시회에 도서 300여 권을 제공하는 서점은 서울 연희동에 위치한 독립 서점 '밤의 서점'입니다. 이 전시회는 서울 성동구에 위치한 복합문화공간 '코사이어티(Cociety)'에서 열립니다 [[ref5]].<|im_end|>

--
예측: 
안산벚꽃음악회가 열리는 인근